# Trajectories of role + trait space

In [ ]:
import json
import os
import sys
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

import re

sys.path.append('.')
sys.path.append('..')

from utils.probing_utils import *


INFO 11-07 14:05:00 [__init__.py:216] Automatically detected platform cuda.


In [2]:
# load in PCA results
CHAT_MODEL_NAME = "Qwen/Qwen3-32B"
#BASE_MODEL_NAME = "google/gemma-2-27b"
model_readable = "Qwen 3 32B"
model_short = "qwen-3-32b"
layer = 32

auditor_model = "sonnet-4.5"

In [ ]:
# filename = "auto40"
components = 1

In [51]:
role_dir = "roles_240" 
trait_dir = "traits_240"

acts_output_dir = f"/workspace/{model_short}/conversations"
acts_input_dir = f"/workspace/{model_short}/dynamics/{auditor_model}/default/activations"
plot_output_dir = f"/root/git/plots/{model_short}/trajectory/{auditor_model}"
os.makedirs(acts_output_dir, exist_ok=True)
os.makedirs(plot_output_dir, exist_ok=True)

## IF NEEDED: Get conversation activations

In [4]:
filename = "spiral"
conversation_file = f"/root/git/persona-subspace/dynamics/results/qwen-3-32b/interactive/{filename}.json"
conversation_obj = json.load(open(conversation_file))
conversation = conversation_obj['conversation']

In [5]:
model, tokenizer = load_model(CHAT_MODEL_NAME)

INFO:accelerate.utils.modeling:Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 3111649280 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [6]:
chat_kwargs = {}
if model_short == "qwen-3-32b":    
    chat_kwargs['enable_thinking'] = False


In [7]:
mean_acts_per_turn = mean_all_turn_activations(model, tokenizer, conversation, layer=layer, chat_format=False, **chat_kwargs)

ValueError: text input must be of type `str` (single example), `list[str]` (batch or single pretokenized example) or `list[list[str]]` (batch of pretokenized examples).

In [57]:
print(len(mean_acts_per_turn))
mean_acts_per_turn = [act.squeeze(0) for act in mean_acts_per_turn]
print(mean_acts_per_turn[0].shape)

46
torch.Size([5120])


In [ ]:
# save activations 
result = {}
result['object'] = conversation_obj
result['activations'] = mean_acts_per_turn
torch.save(result, f"{acts_output_dir}/{filename}.pt")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=torch.bfloat16,
        device_map="auto"  # Use all GPUs
    )
base_model.eval()

Loading checkpoint shards:   0%|          | 0/24 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 4608, padding_idx=0)
    (layers): ModuleList(
      (0-45): 46 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=4608, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4608, out_features=2048, bias=False)
          (v_proj): Linear(in_features=4608, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4608, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=4608, out_features=36864, bias=False)
          (up_proj): Linear(in_features=4608, out_features=36864, bias=False)
          (down_proj): Linear(in_features=36864, out_features=4608, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((4608,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((4608,), eps=1e-06)
        (pre_feedforward_layernorm): G

In [11]:
base_mean_acts_per_turn = mean_all_turn_activations(base_model, tokenizer, conversation, layer=layer, **chat_kwargs)
base_result = {}
base_result['object'] = conversation_obj
base_result['activations'] = base_mean_acts_per_turn
torch.save(base_result, f"{acts_output_dir}/{filename}_base.pt")

In [ ]:
activations_obj = torch.load(f"/workspace/{model_short}/conversations/{filename}.pt", weights_only=False, map_location="cpu")
mean_acts_per_turn = activations_obj['activations']

torch.Size([36, 4608])


In [19]:
contrast_vector = torch.stack(mean_acts_per_turn) - base_mean_acts_per_turn
print(contrast_vector.shape)

torch.Size([36, 4608])


## Cosine similarity with trait and role PCs

In [58]:
# load in activations
role_results = torch.load(f"/workspace/{model_short}/{role_dir}/pca/layer{layer}_pos23.pt", weights_only=False)
trait_results = torch.load(f"/workspace/{model_short}/{trait_dir}/pca/layer{layer}_pos-neg50.pt", weights_only=False)


In [18]:
def pc_cosine_similarity(mean_acts_per_turn, pca_results, n_pcs=8):
    if isinstance(mean_acts_per_turn, list):
        stacked_acts = torch.stack(mean_acts_per_turn)
    else:
        stacked_acts = mean_acts_per_turn
    normalized_acts = F.normalize(stacked_acts, dim=1)
    normalized_pcs = pca_results['pca'].components_[:n_pcs] / np.linalg.norm(pca_results['pca'].components_[:n_pcs], axis=1, keepdims=True)
    cosine_sims = normalized_acts.float().numpy() @ normalized_pcs.T
    return cosine_sims

def pc_projection(mean_acts_per_turn, pca_results, n_pcs=8):
    if isinstance(mean_acts_per_turn, list):
        stacked_acts = torch.stack(mean_acts_per_turn)
    else:
        stacked_acts = mean_acts_per_turn
    stacked_acts = stacked_acts.float().numpy()
    scaled_acts = pca_results['scaler'].transform(stacked_acts)
    projected_acts = pca_results['pca'].transform(scaled_acts)
    return projected_acts[:, :n_pcs]
    
    

In [59]:
role_sims = pc_cosine_similarity(mean_acts_per_turn, role_results, components)
trait_sims = pc_cosine_similarity(mean_acts_per_turn, trait_results, components)
print(role_sims.shape)

role_projs = pc_projection(mean_acts_per_turn, role_results, components)
trait_projs = pc_projection(mean_acts_per_turn, trait_results, components)
print(role_projs.shape)


(46, 6)
(46, 6)


In [ ]:
print(conversation[0])

## Plot trajectory

In [60]:
def plot_mean_response_trajectory(similarity_matrix, conversation=None, title=None, pc_titles=None, projection=False, include_user=False):
    """
    Create a single line plot showing mean response per turn.
    
    Parameters:
    - similarity_matrix: Numpy matrix of shape (n_turns, n_pcs)
    - conversation: Optional conversation data for turn context
    - title: Optional custom title
    - pc_titles: Optional list of PC titles
    - projection: Whether this is projection or cosine similarity
    - include_user: If True, use different marker shapes for even turns (user turns)
    
    Returns:
    - Plotly figure object
    """
    
    print("Creating mean response trajectory plot...")
    
    # Get dimensions
    n_turns, n_pcs = similarity_matrix.shape
    turn_indices = np.arange(n_turns)
    
    # Create default PC titles if not provided
    if pc_titles is None:
        pc_titles = [f"PC{i+1}" for i in range(n_pcs)]
    
    # Define color palette for PCs
    pc_colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', 
        '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
        '#bcbd22', '#17becf', '#aec7e8', '#ffbb78'
    ]
    
    # Helper function to wrap text for hover display
    def wrap_text(text, max_chars_per_line=70):
        """Wrap text to specified line length with HTML breaks."""
        if len(text) <= max_chars_per_line:
            return text
        
        words = text.split()
        lines = []
        current_line = []
        current_length = 0
        
        for word in words:
            if current_length + len(word) + len(current_line) > max_chars_per_line:
                if current_line:  # Don't add empty lines
                    lines.append(' '.join(current_line))
                current_line = [word]
                current_length = len(word)
            else:
                current_line.append(word)
                current_length += len(word)
        
        if current_line:  # Add the last line
            lines.append(' '.join(current_line))
        
        return '<br>'.join(lines)
    
    # Create enhanced turn context for hover text
    turn_contexts = []
    if conversation is not None:
        if include_user:
            # Include both user and assistant turns
            for turn_idx in range(n_turns):
                if turn_idx < len(conversation):
                    turn = conversation[turn_idx]
                    role = turn['role']
                    content = turn['content']
                    
                    # Format hover text
                    hover_parts = [f"<b>Turn {turn_idx} ({role.capitalize()})</b>"]
                    
                    # Truncate content to reasonable length and wrap
                    content_truncated = content[:300] + "..." if len(content) > 300 else content
                    content_wrapped = wrap_text(content_truncated, 70)
                    hover_parts.append(f"<b>{role.capitalize()}:</b> {content_wrapped}")
                    
                    turn_contexts.append('<br>'.join(hover_parts))
                else:
                    turn_contexts.append(f"<b>Turn {turn_idx}</b>")
        else:
            # Original logic - assistant turns only
            assistant_turns = [i for i, turn in enumerate(conversation) if turn['role'] == 'assistant']
            for turn_idx in range(n_turns):
                if turn_idx < len(assistant_turns):
                    conv_turn_idx = assistant_turns[turn_idx]
                    if conv_turn_idx < len(conversation):
                        # Get assistant response
                        assistant_content = conversation[conv_turn_idx]['content']
                        
                        # Get preceding user question (if exists)
                        user_content = ""
                        if conv_turn_idx > 0 and conversation[conv_turn_idx - 1]['role'] == 'user':
                            user_content = conversation[conv_turn_idx - 1]['content']
                        
                        # Format hover text with both user question and assistant response
                        hover_parts = [f"<b>Turn {turn_idx}</b>"]
                        
                        if user_content:
                            # Truncate user content to reasonable length and wrap
                            user_truncated = user_content[:200] + "..." if len(user_content) > 200 else user_content
                            user_wrapped = wrap_text(user_truncated, 70)
                            hover_parts.append(f"<b>User:</b> {user_wrapped}")
                        
                        # Show more of the assistant response (150-200 chars) and wrap
                        assistant_truncated = assistant_content[:300] + "..." if len(assistant_content) > 180 else assistant_content
                        assistant_wrapped = wrap_text(assistant_truncated, 70)
                        hover_parts.append(f"<b>Assistant:</b> {assistant_wrapped}")
                        
                        turn_contexts.append('<br>'.join(hover_parts))
                    else:
                        turn_contexts.append(f"<b>Turn {turn_idx}</b>")
                else:
                    turn_contexts.append(f"<b>Turn {turn_idx}</b>")
    else:
        turn_contexts = [f"<b>Turn {turn_idx}</b>" for turn_idx in range(n_turns)]
    
    # Create Plotly figure
    fig = go.Figure()
    
    # Add line traces for each PC with markers
    for pc_idx in range(n_pcs):
        pc_name = pc_titles[pc_idx]
        similarities = similarity_matrix[:, pc_idx]
        color = pc_colors[pc_idx % len(pc_colors)]
        
        if include_user:
            # Create separate traces for user and assistant turns with different markers
            assistant_indices = turn_indices[1::2]  # Odd indices (1, 3, 5, ...)
            user_indices = turn_indices[::2]        # Even indices (0, 2, 4, ...)
            
            assistant_sims = similarities[1::2]
            user_sims = similarities[::2]
            
            assistant_contexts = [turn_contexts[i] for i in assistant_indices if i < len(turn_contexts)]
            user_contexts = [turn_contexts[i] for i in user_indices if i < len(turn_contexts)]
            
            # Assistant traces (circles with lines)
            fig.add_trace(go.Scatter(
                x=assistant_indices,
                y=assistant_sims,
                mode='lines+markers',
                name=f"{pc_name} (Assistant)",
                line=dict(color=color, width=2),
                marker=dict(color=color, size=6, opacity=0.8, symbol='circle'),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                             '%{text}<br>' +
                             '<b>Projection:</b> %{y:.3f}<br>' +
                             '<extra></extra>',
                text=assistant_contexts,
                legendgroup=f"pc{pc_idx}",
                showlegend=True
            ))
            
            # User traces (squares with lines)
            fig.add_trace(go.Scatter(
                x=user_indices,
                y=user_sims,
                mode='lines+markers',
                name=f"{pc_name} (User)",
                line=dict(color=color, width=2, dash='dash'),  # Dashed lines for users
                marker=dict(color=color, size=6, opacity=0.6, symbol='square'),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                             '%{text}<br>' +
                             '<b>Projection:</b> %{y:.3f}<br>' +
                             '<extra></extra>',
                text=user_contexts,
                legendgroup=f"pc{pc_idx}",
                showlegend=True
            ))
        else:
            # Original single trace per PC
            fig.add_trace(go.Scatter(
                x=turn_indices,
                y=similarities,
                mode='lines+markers',
                name=pc_name,
                line=dict(color=color, width=2),
                marker=dict(color=color, size=4, opacity=0.8),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                             '%{text}<br>' +
                             '<b>Projection:</b> %{y:.3f}<br>' +
                             '<extra></extra>',
                text=turn_contexts
            ))
    
    # Update layout
    default_title = f"Mean Response PC Trajectory"
    if projection:
        yaxis_title = "PC Projection"
    else:
        yaxis_title = "Cosine Similarity with PC"
    
    fig.update_layout(
        title=dict(
            text=title if title else default_title,
            x=0.5,
            font=dict(size=16),
            subtitle={"text": f"{model_readable}, Layer {layer}"}
        ),
        xaxis_title="Conversation Turn",
        yaxis_title=yaxis_title,
        width=1400,
        height=600,
        hovermode='closest',
        legend=dict(
            yanchor="middle",
            y=0.5,
            xanchor="left",
            x=1.02,
            bgcolor="rgba(255,255,255,0.8)"
        )
    )
    
    # Add grid for easier reading
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgray',
        zeroline=True,
        tick0=0,
        dtick=1  # Show every turn
    )
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True)
    
    # Add light vertical lines between turns for clarity
    for turn_idx in range(1, n_turns):
        fig.add_vline(
            x=turn_idx - 0.5,
            line_dash="dot",
            line_color="lightgray",
            line_width=1,
            opacity=0.3
        )
    
    print(f"Created trajectory plot with {n_pcs} PC lines across {n_turns} turns")
    
    return fig

In [ ]:
# Create role PC titles
if model_short == "qwen-3-32b":
    role_pc_titles = ['Assistant-like ↔ role-playing', "mystical/transcendent ↔ mundane/irreverent", "empathetic/vulnerable ↔ analytical/predatory", "concrete/practical ↔ abstract/ideological", "thinking/passive ↔ doing/active", "creative/expressive ↔ rigid/constrained"]
elif model_short == "gemma-2-27b":
    role_pc_titles = ['Assistant-like ↔ role-playing', "inhuman ↔ human", "independent ↔ dependent", "nurturing ↔ adversarial", "social ↔ technical", "structured ↔ liminal"]
elif model_short == "llama-3.3-70b":
    role_pc_titles = ['Assistant-like ↔ role-playing', "mystical ↔ mundane", "empathetic ↔ analytical", "adversarial ↔ nurturing", "provocative ↔ submissive", "social ↔ technical"]


In [ ]:
# Create trait PC titles
if model_short == "qwen-3-32b":
    trait_pc_titles = ["expressive/irreverent ↔ controlled/professional", "analytical ↔ intuitive", "accessible/practical ↔ esoteric/complex", "active/flexible ↔ passsive/rigid", "questioning ↔ confident", "indirect/diplomatic ↔ direct/assertive"]
elif model_short == "gemma-2-27b":
    trait_pc_titles = ["benevolent ↔ evil", "analytical ↔ emotional", "accommodating ↔ confrontational", "grounded ↔ mystical", "thoughtful ↔ decisive", "conformist ↔ defiant"]
elif model_short == "llama-3.3-70b":
    trait_pc_titles = ["agreeable ↔ antagonistic", "emotional ↔ analytical", "bombastic ↔ accessible", "assertive ↔ circumspect", "exploratory ↔ absolutist", "independent ↔ anxious"]


In [63]:
# Create a combined plot showing both role and trait trajectories

# Set include_user parameter - change this to True to show user turns with different markers

def combined_fig(role_projs, trait_projs, title, conversation=None, include_user=False):

    # Create subplot figure
    combined_fig = sp.make_subplots(
        rows=2, cols=1,
        subplot_titles=("Trajectory in Role PC Space", "Trajectory in Trait PC Space"),
        vertical_spacing=0.1
    )

    for tr in plot_mean_response_trajectory(
            role_projs, conversation=conversation, title="<b>Role PCs</b>", pc_titles=role_pc_titles, projection=True, include_user=include_user
        ).data:
        tr.update(
            showlegend=True,
            legendgroup="role",
            legendgrouptitle_text="Role PCs"
        )
        combined_fig.add_trace(tr, row=1, col=1)

    # Add trait traces to second subplot
    for tr in plot_mean_response_trajectory(
            trait_projs, conversation=conversation, title="<b>Trait PCs</b>", pc_titles=trait_pc_titles, projection=True, include_user=include_user
        ).data:
        tr.update(
            showlegend=True,
            legendgroup="trait",
            legendgrouptitle_text="Trait PCs"
        )
        combined_fig.add_trace(tr, row=2, col=1)


    combined_fig.update_layout(
        height=800,
        width=1200,
        title={
            "text": title,
            "subtitle": {"text": f"{model_readable}, Layer {layer}"}
        },
        showlegend=True,
        legend=dict(
            traceorder="grouped",     # keeps groups together
            groupclick="toggleitem",
            x=1.02, xanchor="left",   # place legend outside on the right
            y=1, yanchor="top"
        ),
        legend_tracegroupgap=12       # space between groups
    )
    # Bottom x-axis label only
    combined_fig.update_xaxes(title_text="Conversation Turn", row=2, col=1)

    # Y-axis labels for both subplots
    combined_fig.update_yaxes(title_text="PC Projection", row=1, col=1)
    combined_fig.update_yaxes(title_text="PC Projection", row=2, col=1)
    return combined_fig



In [65]:
fig = combined_fig(role_projs, trait_projs, f"Conversation Trajectories in Persona Subspace: {filename}", conversation=conversation, include_user=True)
fig.show()
fig.write_html(f"{plot_output_dir}/{filename}.html")




Creating mean response trajectory plot...
Created trajectory plot with 6 PC lines across 46 turns
Creating mean response trajectory plot...
Created trajectory plot with 6 PC lines across 46 turns
